In [14]:
import mne 
import pickle
import argparse
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel


from src.params import SUBJ_CLEAN, RESULT_PATH, ACTIVE_RUN, PASSIVE_RUN, FIG_PATH, FREQS_LIST, PREPROC_PATH
from src.utils import get_bids_file
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, ShuffleSplit, RandomizedSearchCV, GridSearchCV, LeaveOneGroupOut
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

from mne.decoding import (
    SlidingEstimator,
    GeneralizingEstimator,
    Scaler,
    cross_val_multiscore,
    LinearModel,
    get_coef,
    Vectorizer,
    CSP,
)

In [15]:
task = 'LaughterActive'
subj_list = SUBJ_CLEAN

if task == 'LaughterActive' :
    run_list = ACTIVE_RUN
elif task == 'LaughterPassive':
    run_list = PASSIVE_RUN

# Select what conditions to compute (str)
cond1 = 'LaughReal'
cond2 = 'LaughPosed'
freq_name = 'alpha'
conditions = cond1 + '-' + cond2
condition_list = [cond1, cond2]
picks = "meg" # Select MEG channels
time_window = True

tmin = 0
tmax = 0.5

- Compute hilbert on specific time-window

In [20]:
def time_window_hilbert(subj_list, RUN_LIST, tmin, tmax, cond1, cond2, FREQS_NAMES, FREQS_LIST, save=True) :
    
    print('Time window : ' + str(tmin) + '-' + str(tmax))
    epochs_list = []

    y_run_cond1 = []
    y_run_cond2 = []

    X_subj = []
    y_subj = []
    
    X_cond1 = []
    X_cond2 = []
    
    power_list = []
    FREQS = [x for x in range(len(FREQS_LIST))]
    idx = 0 #TODO : Change that
    
    group = []

    for l_freq, h_freq in FREQS_LIST :
        print('Processing freqs -->', l_freq, h_freq)
        
        for i_subj, subj in enumerate(subj_list):
            print("=> Process task :", task, 'subject', subj)

            sub_id = 'sub-' + subj
            subj_path = os.path.join(RESULT_PATH, 'meg', 'reports', sub_id)

            if not os.path.isdir(subj_path):
                os.mkdir(subj_path)
                print('BEHAV folder created at : {}'.format(subj_path))
            else:
                print('{} already exists.'.format(subj_path))

            # Take ica data
            _, ica_path = get_bids_file(PREPROC_PATH, stage='ica', subj=subj)
            ica = mne.preprocessing.read_ica(ica_path)
        
            # Select rejection threshold associated with each run
            # TODO : apply threshold per run
            print('Load reject threshold')
            _, reject_path = get_bids_file(RESULT_PATH, stage="AR_epo", task=task, subj=subj, measure='log')

            with open(reject_path, 'rb') as f:
                reject = pickle.load(f)  

            # Select event_id appropriate
            for run in RUN_LIST :
                print("=> Process run :", run)
                        
                if task == 'LaughterPassive' :
                    if run == '02' or run == '03' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12, 'EnvReal' : 21, 
                                    'EnvPosed' : 22}
                    elif run == '04' or run == '05' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12,'ScraReal' : 31, 
                                    'ScraPosed' : 32,}
                if task == 'LaughterActive' :
                        event_id = {'LaughReal' : 11, 'LaughPosed' : 12, 'Good' : 99, 'Miss' : 66}


                # Take raw data filtered : i.e. NO ICA 
                print('ICA')
                _, raw_path = get_bids_file(PREPROC_PATH, stage='proc-filt_raw', task=task, run=run, subj=subj)
                raw = mne.io.read_raw_fif(raw_path, preload=True)
                raw_filter = raw.copy()
                raw_filter = ica.apply(raw_filter)

                epochs_psds = []

                freq_name = FREQS_NAMES[idx]

                print('Apply filter')
                info = raw_filter.info
                raw_filter.filter(l_freq, h_freq) # Apply filter of interest
                raw_hilbert = raw_filter.apply_hilbert(envelope=True)

                picks = mne.pick_types(raw.info, meg=True, ref_meg=False, eeg=False, eog=False)

                power_hilbert = raw_hilbert.copy()
                power_hilbert._data = power_hilbert._data**2

                # Segmentation
                print('Segmentation')
                events = mne.find_events(raw)
                epochs = mne.Epochs(
                    power_hilbert,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=(0, 0),
                    picks=picks)    

                # Auto-reject epochs
                epochs.drop_bad(reject=reject)

                if subj == subj_list[0] and run == run_list[0]:
                    head_info = epochs.info['dev_head_t']
                else:
                    epochs.info['dev_head_t'] = head_info

                epochs.equalize_event_counts(['LaughReal', 'LaughPosed'])
                
                for i_cond, cond in enumerate(epochs[cond1, cond2].get_data()) : 
                    group.append(i_subj)
                    
                epochs_list.append(epochs)

                del epochs
                
        epochs_all_run = mne.concatenate_epochs(epochs_list)  

        print(epochs_all_run)
        
        # Compute power (= envelop **2)
        print('Calculte power')
        ave_time_cond1 = np.mean(epochs_all_run[cond1].get_data(), axis=2)
        ave_time_cond2 = np.mean(epochs_all_run[cond2].get_data(), axis=2)
        
        X_cond1.append(ave_time_cond1)
        X_cond2.append(ave_time_cond2)
        print('Cond1 shape :', X_cond1)
        print('Cond2 shape :', X_cond2)
        
        print('Concatenation')
        X_subj.append(np.concatenate((ave_time_cond1, ave_time_cond2)))
        y_subj.append(np.concatenate((epochs_all_run[cond1].events[:, 2], epochs_all_run[cond2].events[:, 2])))
        
        print(X_subj[-1].shape)
        print(y_subj[-1].shape)
        print(np.array(group).shape)
        
        if save == True : 
            stage = 'ml-'+ str(tmin) + '-' + str(tmax)
            sub_name = 'all'
            run_name = 'all'

            save_X = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                                run_name, conditions,
                                                                                                'X-subj', stage)
            save_y = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                                run_name, conditions,'y-subj', stage)

            save_group = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                                run_name, conditions,
                                                                                                'group', stage)

            save_X1 = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                                run_name, conditions,
                                                                                                'X-cond1', stage)

            save_X2 = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                                run_name, conditions,
                                                                                                'X-cond2', stage)

            with open(save_X, 'wb') as f:
                pickle.dump(X_subj, f)

            with open(save_y, 'wb') as f:
                pickle.dump(y_subj, f)

            with open(save_group, 'wb') as f:
                pickle.dump(group, f)

            with open(save_X1, 'wb') as f:
                pickle.dump(X_cond1, f)

            with open(save_X2, 'wb') as f:
                pickle.dump(X_cond2, f)

    return X_subj, y_subj, group, epochs_all_run

In [4]:
def all_time(subj_list, run_list, freq_name):
    print('All time')
    for subj in subj_list: 

        print(subj)

        for run in run_list :

            # importer les psd (enveloppe spectrale)
            _, path_epochs = get_bids_file(RESULT_PATH, task=task, subj=subj, run=run, measure=freq_name, stage="psd_epo")
            epochs = mne.read_epochs(path_epochs, verbose=None)

            epochs.apply_baseline(baseline=(None, 0))
            epochs.resample(50)

            if subj == subj_list[0] and run == run_list[0]:
                head_info = epochs.info['dev_head_t']
            else:
                epochs.info['dev_head_t'] = head_info

            epochs_list.append(epochs)

        epochs_all_run = mne.concatenate_epochs(epochs_list)  

        # Compute power (= envelop **2)
        power_cond1 = epochs_all_run[cond1].get_data()**2 # Shape n_epochs, n_chan, n_times
        power_cond2 = epochs_all_run[cond2].get_data()**2 # Shape n_epochs, n_chan, n_times
        
        X_subj.append(np.concatenate((power_cond1, power_cond2)))
        y_subj.append(np.concatenate((epochs_all_run[cond1].events[:, 2], epochs_all_run[cond2].events[:, 2])))

        return X_subj, y_subj

In [21]:
# Change this params
epochs_list = []

y_run_cond1 = []
y_run_cond2 = []

X_subj = []
y_subj = []

subj_list = SUBJ_CLEAN
run_list = run_list

if time_window == True :
    X_subj, y_subj, group, epochs_all_run = time_window_hilbert(subj_list, run_list, tmin, tmax, cond1, cond2, FREQS_NAMES=[freq_name], FREQS_LIST=[FREQS_LIST[2]], save=True)
    
else : 
    X_subj, y_subj = all_time(subj_list, run_list, freq_name)


Time window : 0-0.5
Processing freqs --> 8 12
=> Process task : LaughterActive subject 01
/home/claraelk/scratch/laughter_data/results/meg/reports/sub-01 already exists.
Reading /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_ica.fif ...
Now restoring ICA solution ...
Ready.
Load reject threshold
=> Process run : 07
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-07_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (189 components)
    Zeroing out 8 ICA components
    Projecting back using 273 PCA components
Apply filter
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 302 out of 302 | elapsed:    2.0s finished


Segmentation
Trigger channel has a non-zero initial value of 68 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
74 events found
Event IDs: [ 5 11 12 66 99]
Not setting metadata
49 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 49 events and 601 original time points ...
0 bad epochs dropped
Dropped 5 epochs: 0, 1, 3, 5, 15
Using data from preloaded Raw for 20 events and 601 original time points ...
=> Process run : 08
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-08_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.99

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 302 out of 302 | elapsed:    2.0s finished


Segmentation
Trigger channel has a non-zero initial value of 68 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
75 events found
Event IDs: [ 5 11 12 66 99]
Not setting metadata
50 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 50 events and 601 original time points ...
0 bad epochs dropped
Dropped 5 epochs: 0, 2, 4, 18, 20
Using data from preloaded Raw for 20 events and 601 original time points ...
=> Process run : 09
ICA
Opening raw data file /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_run-09_proc-filt_raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 299999 =      0.000 ...   249.999 secs
Ready.
Current compensation grade : 3
Reading 0 ... 299999  =      0.000 ...   249.9

KeyboardInterrupt: 

In [16]:
# Take data

stage = 'ml-'+ str(tmin) + '-' + str(tmax)
sub_name = 'all'
run_name = 'all'

save_X = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'X-subj', stage)
save_y = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,'y-subj', stage)

save_group = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'group', stage)

save_X1 = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'X-cond1', stage)

save_X2 = RESULT_PATH + 'meg/reports/sub-all/ml/sub-{}_task-{}_run-{}_cond-{}_meas-{}_{}.pkl'.format(sub_name, task, 
                                                                                    run_name, conditions,
                                                                                    'X-cond2', stage)

    

with open(save_X, 'rb') as f:
    X_subj = pickle.load(f)
    
with open(save_y, 'rb') as f:
    y_subj = pickle.load(f)
    
with open(save_group, 'rb') as f:
    group = pickle.load(f)
    
with open(save_X1, 'rb') as f:
    X_cond1 = pickle.load(f)

with open(save_X2, 'rb') as f:
    X_cond2 = pickle.load(f)
    

## ML single feature
- with time windows

In [5]:
CHAN = np.arange(0, 270, 1)

all_y_pred = []

# Select the classifier & cross-val method
X = X_subj[-1] #X_subj[-1]
print(X.shape)

y = y_subj[-1]
print(y.shape)

groups = np.array(group)

# TODO : Separate in X_train and y_train, X_test, y_test
for chan in CHAN :
    print('-->Process channel :', chan)
    
    y_pred = []
    
    parameters = {'kernel':('linear', 'rbf',  'rbf', 'sigmoid'), 'C':[1, 10]}
    clf = svm.SVC()
    
    cv = LeaveOneGroupOut() # Cross-validation via LOGO
    
    # Select channel of interest
    X_chan = X[:, chan]
    X_classif = X_chan.reshape(-1, 1) # Reshape (n_event, 1)
    print(X_classif.shape)
    
    for train, test in cv.split(X_classif, y, groups) : #Split in train-test with LOGO 
        print('Split')
        
        # Find best params with GridSearch
        print('---->Find best params')
        gridsearch = GridSearchCV(clf, parameters, cv=cv, verbose=True).fit(X=X_classif[train], 
                                                          y=y[train], 
                                                          groups=groups[train])
        
        # Select best params
        best_params = gridsearch.best_estimator_
        print('Best params : ' + str(best_params))
        clf = best_params # Use classifier with best params
        
        # Predict model
        print('---->Predict model')
        
        y_pred_split = clf.predict(X_classif[test]) # Predict y 
        print(accuracy_score(y_pred_split, y[test])) # Show accuracy
        
        y_pred.append(accuracy_score(y_pred_split, y[test]))
    
    print('Average accuracy for chan: ', chan)
    y_pred_ave = np.mean(np.array(y_pred))
    all_y_pred.append(y_pred_ave)
    print(y_pred_ave)        
    

(3240, 270)
(3240,)
-->Process channel : 0
(3240, 1)
Split
---->Find best params
Best params : SVC(C=10, kernel='sigmoid')
---->Predict model
0.475
Split
---->Find best params
Best params : SVC(C=1, kernel='sigmoid')
---->Predict model
0.48333333333333334
Split
---->Find best params
Best params : SVC(C=1, kernel='sigmoid')
---->Predict model
0.4666666666666667
Split
---->Find best params
Best params : SVC(C=1, kernel='sigmoid')
---->Predict model
0.45
Split
---->Find best params
Best params : SVC(C=1, kernel='sigmoid')
---->Predict model
0.55
Split
---->Find best params
Best params : SVC(C=10, kernel='sigmoid')
---->Predict model
0.5416666666666666
Split
---->Find best params
Best params : SVC(C=10, kernel='sigmoid')
---->Predict model
0.525
Split
---->Find best params
Best params : SVC(C=10, kernel='sigmoid')
---->Predict model
0.43333333333333335
Split
---->Find best params
Best params : SVC(C=10, kernel='sigmoid')
---->Predict model
0.49166666666666664
Split
---->Find best params
Be

KeyboardInterrupt: 

In [18]:
CHAN = np.arange(0, 270, 1)

all_scores = []

# Select the classifier & cross-val method
X = X_subj[-1] #X_subj[-1]
print(X.shape)

y = y_subj[-1]
print(y.shape)

groups = np.array(group)

# TODO : Separate in X_train and y_train, X_test, y_test
for chan in CHAN :
    print('-->Process channel :', chan)
    
    y_pred = []
    
    parameters = {'kernel':('linear', 'rbf',  'rbf', 'sigmoid'), 'C':[1, 10]}
    clf = svm.SVC()
    
    cv = LeaveOneGroupOut() # Cross-validation via LOGO
    
    # Select channel of interest
    X_chan = X[:, chan]
    X_classif = X_chan.reshape(-1, 1) # Reshape (n_event, 1)
    print(X_classif.shape)
    
    # Find best params with GridSearch
    # Use RandomSearch
    print('---->Find best params')
    search = RandomizedSearchCV(clf, parameters, cv=cv, verbose=True, n_jobs=-1).fit(X=X_classif, 
                                                                                    y=y, 
                                                                                    groups=groups)
        
    # Select best params
    best_params = search.best_estimator_
    print('Best params : ' + str(best_params))

    # Accuracy score
    scores = search.score(X=X_classif, y=y) 
    print('Scores', scores)
    #print('Best scores', scores.best_score_)
    
    all_scores.append(scores)
    


(3240, 270)
(3240,)
-->Process channel : 0
(3240, 1)
---->Find best params
Fitting 27 folds for each of 8 candidates, totalling 216 fits


/home/claraelk/notebook/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best params : SVC(C=10, kernel='sigmoid')
Scores 0.5064814814814815
-->Process channel : 1
(3240, 1)
---->Find best params
Fitting 27 folds for each of 8 candidates, totalling 216 fits


/home/claraelk/notebook/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best params : SVC(C=10, kernel='sigmoid')
Scores 0.5092592592592593
-->Process channel : 2
(3240, 1)
---->Find best params
Fitting 27 folds for each of 8 candidates, totalling 216 fits


...

Best params : SVC(C=10, kernel='sigmoid')
Scores 0.4993827160493827
-->Process channel : 146
(3240, 1)
---->Find best params
Fitting 27 folds for each of 8 candidates, totalling 216 fits


/home/claraelk/notebook/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best params : SVC(C=10, kernel='sigmoid')
Scores 0.504320987654321
-->Process channel : 147
(3240, 1)
---->Find best params
Fitting 27 folds for each of 8 candidates, totalling 216 fits


/home/claraelk/notebook/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# T-test or cond1-cond2/cond2

for chan in CHAN :

    # conduct t test
    t, p = ttest_rel(X_cond1, X_cond2) # Paired t-test

    # display results
    format_dict = dict(
     t_val=t, p=p
    )
    print(report.format(**format_dict))

## Multi-features
- Decoding over-time

In [ ]:
# We will train the classifier on all left visual vs auditory trials on MEG
# TODO : add feature selection (Channel selection)
sfs = SequentialFeatureSelector(classifier, n_features_to_select=60)

clf = make_pipeline(StandardScaler(), sfs, classifier)

time_decod = SlidingEstimator(clf, scoring="roc_auc", verbose=True, n_jobs=-1)
# here we use cv=3 just for speed
scores = cross_val_multiscore(time_decod, X, y, cv=3, n_jobs=-1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs.times, scores, label="score")
ax.axhline(0.5, color="k", linestyle="--", label="chance")
ax.set_xlabel("Times")
ax.set_ylabel("AUC")  # Area Under the Curve
ax.legend()
ax.axvline(0.0, color="k", linestyle="-")
ax.set_title("Sensor space decoding")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.

  0%|          | Fitting SlidingEstimator : 0/100 [00:00<?,       ?it/s]

  0%|          | Fitting SlidingEstimator : 0/100 [00:00<?,       ?it/s]